# DAC Contest
This reference design will help you walk through a design flow of DAC SDC 2020.

This is a simplified design to help users get started on the FPGA platform and to understand the overall flow. It does not contain any object detection hardware.

If you have any questions, please post on the [Piazza page](piazza.com/dac_2018/winter2020/dacsdc2020/home).

## Hardware
The base hardware design contains the Zynq MPSoC processor (A53) with a DMA and FIFO implemented in the PL.  
Note that in this starting hardware, no actual image processing/detection is done.
Pictures are:
1. sent from PS to DMA, then DMA to FIFO
2. sent back from FIFO to DMA, then DMA to PS.

Note that the FIFO should be replaced with meaningful hardware to process the pictures.

You can create a Vivado project by opening Vivado 2019.1, changing to the `hw` directory, and running `source dac_sdc.tcl`.  This will create and open a project that you can edit.  Each time you submit your files, you should run `File->Project->Write Tcl...` to generate a new tcl file to submit.  Be sure to check the `Recreate Block Designs using Tcl` box, and include any custom IP modules you use.

## Software
Note:
1. Any change in `dac_sdc.py` will make your design fail in evaluation. This file should not be changed.
2. You can use both PS and PL side to do inference.

Timing **should include the entire execution of your design, including reading the image from the SD card**, as shown in the notebook below.

Please check the reference for details. You can exclude "save_results_xml" when counting time.

Batch size is 500 by default.

Please write your results using the methods `write()` and `save_results_xml()`.

It is your choice how to record the inference result. 
However, it must be readable, and you must convert it to XML files. An example is provided.
Please place your code into `teamname.ipynb`. 
Your design should be able to process all evaluation images in five cells.


Your notebook should contain 4 code cells:
  1. Importing all libraries and creating your Team object.
  2. Downloading the overlay, and performany any one-time configuration.
  3. Processing all images.
  4. Writing results to XML and any other cleanup.

The following cells are not a real design for image detection. But you can use them to start your design. (Section 3 is broken into a few cells for demonstration purposes)

### 1. Importing all libraries

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../common"))

import math
import time
import numpy as np
from PIL import Image
from matplotlib import pyplot
import cv2
from datetime import datetime

import pynq
import dac_sdc
from IPython.display import display

team_name = 'sample_team'
team = dac_sdc.Team(team_name, batch_size = 1)

**Your team directory where you can access your bitstream, notebook, and any other files you submit, is available as `team.team_dir`.**


### 2. Preparing the overlay and weight loading
Overlay loading must be executed in this cell.

In this sample hardware, the DMA instance is exposed as an attribute of the overlay object.  You aren't required to use DMA for your hardware solution.


In [ ]:
overlay = pynq.Overlay(team.get_bitstream_path())
dma = overlay.axi_dma_0

Data flows from PS to PL, then back to PS by DMA. Using an interrupt is recommended.

### 3. Processing Images


In this section we first demonstrate how to use then `opencv` library to read and manipuate images, then we show how to send an image to the hardware using DMA.  Finally, we demonstrate how to loop through the images and processing them, recording run-time and energy usage.  The final example will is most closely related to what you need to do.


In [ ]:
interval_time = 0

# Skip image 0
team.get_next_batch()

# Get image 1
image_path = team.get_next_batch()[0]

original_image = Image.open(image_path)
original_array = np.array(original_image)
original_image.close()

original_result = Image.fromarray(original_array)
display(original_result)

old_width, old_height = original_image.size
print("Original image size: {}x{} pixels.".format(old_height, old_width))

We will now resize the image using the openCV library.

In [ ]:
new_width, new_height = int(old_width/2), int(old_height/2)
original_image = Image.open(image_path)
resized_image = original_image.resize((new_width, new_height), 
                                      Image.ANTIALIAS)
resized_array = np.array(resized_image)
original_image.close()

resize_result = Image.fromarray(resized_array)
display(resize_result)

width, height = resized_image.size
print("Resized image size: {}x{} pixels.".format(height, width))

### Pushing the picture through the pipeline
In the following example, we will also use contiguous memory arrays for sending
and receiving data.

The size of the buffer depends on the size of the input or output data.
Since the image we process in the following example (`0.jpg`) has 453x674 RGB pixels,
we will use `cma_array` of the corresponding size as well.

In [ ]:
xlnk = pynq.Xlnk()
in_buffer = xlnk.cma_array(shape=(453, 674, 3), dtype=np.uint8, cacheable = 1)
out_buffer = xlnk.cma_array(shape=(453, 674, 3), dtype=np.uint8, cacheable = 1)

Note: In the following example, we are only dealing with one image. 
We will just send one image to the FIFO and loop it back. 
In the real contest, you should process all the RGB images in every batch.

Note that the `rgb_array` has to be copied into the contiguous memory array
(deep copy).

In [ ]:
interval_time = 0

# Get image 0
team.reset_batch_count()
image_path = team.get_next_batch()[0]

bgr_array = cv2.imread(str(image_path))
rgb_array = cv2.cvtColor(bgr_array, cv2.COLOR_BGR2RGB)
in_buffer[:] = rgb_array

pyplot.imshow(in_buffer)
pyplot.show()

Now we will push the data from input buffer through the pipeline to the output buffer.

In [ ]:
def dma_transfer():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    

dma_transfer()

pyplot.imshow(out_buffer)
pyplot.show()

### Iterating through all images by batch, recording timing and energy measurements

In this cell, you will:
  * Perform any startup configuraton of your hardware (Download weights, etc.)
  * Measure the time and energy for processing all the images. 
  
The timer and power measurements should be running for your entire execution.  The code below shows how you can measure the total energy usage.  Make sure you follow this approach.

In the following example we will time the processing and measure energy usage of a single picture.

In [ ]:
interval_time = 0
total_time = 0
total_energy = 0
result = list()
team.reset_batch_count()
rails = pynq.get_rails()

# Start timer and energy recorder.  If this fails, then you may not have configured
# your system to measure power usage.  Follow the steps provided at
# https://github.com/jgoeders/dac_sdc_2020/blob/master/support/measure_power/README.md
start = time.time()    
recorder = pynq.DataRecorder(rails["5V"].power)
with recorder.record(0.05):        
    while True:
        # get a batch of images
        image_paths = team.get_next_batch()
        if image_paths is None:
            break

        # Read all images in this batch from the SD card.
        # This part doesn't count toward your time/energy usage.
        rgb_imgs = []
        for image_path in image_paths:
            bgr_img = cv2.imread(str(image_path))    
            rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
            rgb_imgs.append(rgb_img)
   
        for img in rgb_imgs:
            in_buffer[:] = img    
            dma_transfer()
            
# timer stop after batch processing is complete
end = time.time()
t = end - start
    
# Energy measurements    
energy = recorder.frame["5V_power"].mean() * t    

total_time = t
total_energy = energy
print("Total time:", total_time, "seconds")
print("Total energy:", total_energy, "J")

### 4. Recording results, cleanup

It is your responsibility to record your results.  You should call `team.save_results_xml(result_list, total_runtime, total_energy)` to save your results.
  * `results_list`: Each element in this list is a 4-integer list [xmin, xmax, ymin, ymax] indicating the bounding box location deterined by your object detection.  This list should have the same number of elements as there are images.  The first element in the list will correspond to image 0.jpg, etc.
  * `total_runtime`: The total runtime of your processing of all images, as described above.
  * `total_energy`: The total energy of your processing of all images, as described above.
  
The box below shows an example:


In [ ]:
# These are fake object locations since the example design doesn't actually perform object detection
result_rectangle =  [[0,358,0,263],[0,1350,0,707]]

team.save_results_xml(result_rectangle, total_time, total_energy)
print("XML results written successfully.")

# Remember to free the contiguous memory after usage.
xlnk.xlnk_reset()

### What to submit

Submit your:
  * Notebook
  * .bit and .hwh files
  * The tcl file that can be used to create your Vivado project in 2019.1
  * Any supporting hardware files that Vivado needs (eg. IP)
  * Any other files your notebook needs to run (eg. weights file)
 